In [1]:
#import files
import numpy as np
import pandas as pd

In [2]:
#Load Dataset
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
#merge data by item id
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 944
Num of Movies: 1682


In [10]:
#Train test split
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

In [9]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [23]:
train_data

,user_id,item_id,rating,timestamp,title
49446,19,211,4,885412840,M*A*S*H (1970)
53510,640,150,4,886474493,Swingers (1996)
68349,184,511,4,889908740,Lawrence of Arabia (1962)
32045,165,318,5,879525961,Schindler's List (1993)
37312,528,484,3,886101695,"Maltese Falcon, The (1941)"
...,...,...,...,...,...
28930,637,280,2,882904679,Up Close and Personal (1996)
52949,488,168,4,891293910,Monty Python and the Holy Grail (1974)
3496,650,29,2,891382877,Batman Forever (1995)
90682,487,45,5,883446725,Eat Drink Man Woman (1994)


In [25]:
#comparing similarity using cosine simularity
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [26]:
user_similarity

array([[0.        , 0.88503052, 0.96071083, ..., 0.87061846, 0.70660194,
        0.8706104 ],
       [0.88503052, 0.        , 0.89243967, ..., 0.93141307, 0.925966  ,
        1.        ],
       [0.96071083, 0.89243967, 0.        , ..., 0.92280144, 0.96414797,
        1.        ],
       ...,
       [0.87061846, 0.93141307, 0.92280144, ..., 0.        , 0.87542865,
        0.79661181],
       [0.70660194, 0.925966  , 0.96414797, ..., 0.87542865, 0.        ,
        0.93253131],
       [0.8706104 , 1.        , 1.        , ..., 0.79661181, 0.93253131,
        0.        ]])

In [27]:
#Predictions based on similarity 
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [28]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

In [29]:
#Evaluation using RMSE, Only predicted ratings in the test dataset are considered, filtering out all other elements in the prediction matrix with prediction[ground_truth.nonzero()]. 
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [30]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.12603209500508
Item-based CF RMSE: 3.4523386706114034


In [31]:
#Item-Item Model Based Collaborative filtering
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


In [32]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.729319037187675
